In [10]:
import os
import numpy as np
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import utils
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor, Normalize, Compose, Grayscale
from torchvision.models import vgg16

In [26]:
transform = Compose([
    ToTensor(),
    Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    #Grayscale(num_output_channels=1)
    
])

# Create the dataset using the ImageFolder class
dataset = ImageFolder("gestures", transform=transform)

# Create the dataloader
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)    


In [33]:
class GestureClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.vgg = vgg16(pretrained=True)
        self.classifier = nn.Sequential(
            nn.Linear(1000, 128),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        x = self.vgg(x)
        x = self.classifier(x)
        return x

In [34]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = GestureClassifier().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

num_epochs = 10

In [35]:
for epoch in range(num_epochs):
    model.train()

    for batch_idx, (data, target) in enumerate(dataloader):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()

        output = model(data)
        loss = criterion(output, target)
        loss.backward()

        optimizer.step()

        #if batch_idx % 100 == 0:
        print('Epoch {} Batch {}: Loss = {}'.format(epoch, batch_idx, loss.item()))

Epoch 0 Batch 0: Loss = 2.4299466609954834
Epoch 0 Batch 1: Loss = 2.462947368621826
Epoch 0 Batch 2: Loss = 2.3326876163482666
Epoch 0 Batch 3: Loss = 2.3505496978759766
Epoch 0 Batch 4: Loss = 2.2758853435516357
Epoch 0 Batch 5: Loss = 2.3100781440734863
Epoch 0 Batch 6: Loss = 2.2490036487579346
Epoch 0 Batch 7: Loss = 2.328005790710449
Epoch 0 Batch 8: Loss = 2.2681498527526855
Epoch 0 Batch 9: Loss = 2.3554751873016357
Epoch 0 Batch 10: Loss = 2.326913595199585
Epoch 0 Batch 11: Loss = 2.211054563522339
Epoch 0 Batch 12: Loss = 2.3182804584503174
Epoch 0 Batch 13: Loss = 2.2465829849243164
Epoch 0 Batch 14: Loss = 2.3135905265808105
Epoch 0 Batch 15: Loss = 2.268874406814575
Epoch 0 Batch 16: Loss = 2.2443597316741943
Epoch 0 Batch 17: Loss = 2.276073455810547
Epoch 0 Batch 18: Loss = 2.281475782394409
Epoch 0 Batch 19: Loss = 2.2574994564056396
Epoch 0 Batch 20: Loss = 2.2227087020874023
Epoch 0 Batch 21: Loss = 2.293374538421631
Epoch 0 Batch 22: Loss = 2.2282180786132812
Epoch 

KeyboardInterrupt: 

In [37]:
model.eval()

for i, (data, labels) in enumerate(dataloader):
    data, labels = data.to(device), labels.to(device)
    with torch.no_grad():
        output = model(data)
        _, predicted = torch.max(output, 1)
        correct = torch.sum(predicted == labels).item()
        print('Accuracy batch {}: {}'.format(i, correct / len(labels)))

Accuracy batch 0: 1.0
Accuracy batch 1: 1.0
Accuracy batch 2: 0.96875
Accuracy batch 3: 1.0
Accuracy batch 4: 1.0
Accuracy batch 5: 0.9375
Accuracy batch 6: 1.0
Accuracy batch 7: 1.0
Accuracy batch 8: 1.0
Accuracy batch 9: 1.0
Accuracy batch 10: 1.0
Accuracy batch 11: 1.0
Accuracy batch 12: 0.96875
Accuracy batch 13: 1.0
Accuracy batch 14: 1.0
Accuracy batch 15: 0.96875
Accuracy batch 16: 1.0
Accuracy batch 17: 1.0
Accuracy batch 18: 1.0
Accuracy batch 19: 1.0
Accuracy batch 20: 1.0
Accuracy batch 21: 1.0
Accuracy batch 22: 1.0
Accuracy batch 23: 0.96875
Accuracy batch 24: 0.96875
Accuracy batch 25: 0.96875


KeyboardInterrupt: 